In [1]:
#import modules
import pandas as pd
import numpy as np
from datetime import datetime
#See max columns in a dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [2]:
# Read the CSV file with specified data types and skipped first 4 rows
vacant_lots_df = pd.read_csv("data/southern_dallas_progress_export_20230906.csv")



In [3]:
#To see the rows before dropping duplicates
vacant_lots_df.shape

(39645, 26)

In [4]:
# Remove duplicates based on all columns
vacant_lots_df = vacant_lots_df.drop_duplicates(subset=['Parcel ID'])


In [5]:
#Rows after duplicates have been dropped
vacant_lots_df.shape

(33941, 26)

In [6]:
# Recode 0 as NA in the "Year Built" column
vacant_lots_df['Year Built'] = np.where(vacant_lots_df['Year Built'] == 0, np.nan, vacant_lots_df['Year Built'])

In [7]:
#Creates field Property Age Years in df

#Assign a variable to the current year
current_year = 2023

# Calculate "Property Age Years" as the difference between 2023 and Year Built
vacant_lots_df['Property Age Years'] = current_year - vacant_lots_df['Year Built']

# Assign the calculated columns to variables for potential future use
property_age_years = vacant_lots_df['Property Age Years']


In [8]:
#Creates field Vacancy Days in df

# Convert today's date to a Timestamp object
today = pd.Timestamp(datetime.today().date())

# Convert the 'Vacant Since' column to a Datetime column 
vacant_lots_df['Vacant Since'] = pd.to_datetime(vacant_lots_df['Vacant Since'])

# Calculate "Vacancy Days" as the difference between today's date and "Vacant Since" date
vacant_lots_df['Vacancy Days'] = (today - vacant_lots_df['Vacant Since']).dt.days

# Assign the calculated columns to variables for potential future use
vacancy_days = vacant_lots_df['Vacancy Days']

In [9]:

#For ending set of 3 questions use a function to map Single Family,Multi, and Commercial
def zoning_triple_category(row):
    zoning=row['Zoning']
    
    if pd.notna(zoning):#check if thed value is not NaN
        if "Single Family" in zoning or "Mobile Home" in zoning:
            return "Single Family"
        elif any(keyword in zoning for keyword in ["Multifamily", "Duplex", "Cluster Housing", "Townhouse"]):
            return "Multifamily"
        elif "Commercial" in zoning or "commercial" in zoning:
            return "Commercial"
        return "Other" #Return "Other" for NaN or missing values
vacant_lots_df['Zoning Category 3'] = vacant_lots_df.apply(zoning_triple_category, axis=1)  

In [10]:
# Check if "CHURCH" is a partial match in the "Current Use" column and create the "Owner is Church" column
vacant_lots_df['Owner is Church']=vacant_lots_df['Current Use'].str.contains('CHURCH', case=False, na=True)

# Map the True/False values to 'Yes' and 'No' in the "Owner is Church" column
vacant_lots_df['Owner is Church'] = vacant_lots_df['Owner is Church'].map({True: 'Yes', False: 'No'})

owner_is_church=vacant_lots_df["Owner is Church"]

In [11]:

#Checks strings in zoning column and maps them
def classify_zoning_category(row):
    zoning = row['Zoning']
    
    if pd.notna(zoning):  # Check if the value is not NaN
        if "Central Area" in zoning:
            return "Central Area"
        elif "Retail" in zoning or "Neighborhood Service" in zoning:
            return "Retail"
        elif any(keyword in zoning for keyword in ["Single Family", "Multifamily", "Duplex", "Cluster Housing", "Townhouse"]):
            return "Residential"
        elif "Office" in zoning:
            return "Office"
        elif "Industrial" in zoning:
            return "Industrial"
        elif "Commercial" in zoning or "commercial" in zoning:  # Handle the typo
            return "Commercial"
    return "Other"  # Return "Other" for NaN or missing values

# Apply the function to create the "Zoning Category" column
vacant_lots_df['Zoning Category'] = vacant_lots_df.apply(classify_zoning_category, axis=1)

In [12]:
# Define a function to apply the classification logic
def classify_owner(row):
    if pd.notna(row['Public Owner']):
        return row['Public Owner']
    elif row['Public Owner'] == 'City of Dallas':
        return 'City of Dallas'
    elif row['Public Owner'] == 'City of Dallas Housing Authority':
        return 'City of Dallas Housing Authority'
    elif row['Public Owner'] == 'County of Dallas':
        return 'County of Dallas'  
    elif row['Owner is Church'] == 'Yes':
        return 'Church'
    elif row['Owner is Business'] == 'Yes':
        return 'Business'
    elif row['Bank Owned Property'] == 'Yes':
        return 'Bank'
    else:
        return 'Private Owner'

# Apply the function to create the "Owner" column
vacant_lots_df['Owner'] = vacant_lots_df.apply(classify_owner, axis=1)
   

In [13]:
# Selecting columns of interest and filtering by 'Property Type' == 'Structure'
structure_df = vacant_lots_df[vacant_lots_df['Property Type'] == 'Structure']

# Selecting columns of interest and filtering by 'Property Type' == 'Land' for land dataframe
land_df = vacant_lots_df[vacant_lots_df['Property Type'] == 'Land']

# Filter residential properties
residential_properties = vacant_lots_df[vacant_lots_df['Zoning Category'] == 'Residential']

# Filter commercial properties used Zone Category 3 column for zoned single family residential, commercial and multifamily questions
commercial_properties = vacant_lots_df[vacant_lots_df['Zoning Category 3'] == 'Commercial']


In [14]:
vacant_lots_df.head()

,Parcel ID,Adddress,Owner,Is Vacant Property,Property Type,Current Use,Year Built,Average Year Built By Council District,Owner Location,Owner Property Count,Public Owner,City or County Owned,Owner is Business,Owner Occupied,Suspected Heir Property,Bank Owned Property,Taxes Delinquent Since,Delinquent Taxes,Is Tax Delinquent,Code Liens - Count,Code Liens - Amount,Zip Code,Zoning,Council District,Vacant Since,Land Size,Property Age Years,Vacancy Days,Zoning Category 3,Owner is Church,Zoning Category
0,1.00561E+11,"1109 WOOD ST, DALLAS, TX",Business,Yes,Land,C12 - COMMERCIAL - VACANT PLOTTED LOTS/TRACTS ...,NaN,1956.0,In City,20,NaN,No,Yes,No,No,No,NaN,NaN,No,NaN,NaN,75202,Central Area - CA-1(A)-Central Area,District 14,2022-04-01,0.124653,NaN,524.0,Other,No,Central Area
1,1.01971E+11,"1908 ELM ST, DALLAS, TX",Private Owner,Yes,Structure,C12 - COMMERCIAL - VACANT PLOTTED LOTS/TRACTS ...,NaN,1956.0,In City,1,NaN,No,No,Unknown,No,No,NaN,NaN,No,NaN,NaN,75201,Other - PD-619,District 14,2023-06-01,0.122321,NaN,98.0,Other,No,Other
2,1.01995E+11,"1913 COMMERCE ST, DALLAS, TX",City of Dallas,Yes,Land,C12 - COMMERCIAL - VACANT PLOTTED LOTS/TRACTS ...,NaN,1956.0,In City,2926,City of Dallas,Yes,No,No,No,No,NaN,NaN,No,NaN,NaN,75201,Other - PD-619,District 14,2022-04-01,0.117204,NaN,524.0,Other,No,Other
3,1.03702E+11,"2523 MAIN ST, DALLAS, TX",Business,Yes,Land,C12 - COMMERCIAL - VACANT PLOTTED LOTS/TRACTS ...,NaN,1925.0,In City,90,NaN,No,Yes,No,No,No,NaN,NaN,No,NaN,NaN,75226,Other - PD-269,District 2,2022-04-01,0.126854,NaN,524.0,Other,No,Other
4,1.0372E+11,"2619 MAIN ST, DALLAS, TX",Business,Yes,Land,C12 - COMMERCIAL - VACANT PLOTTED LOTS/TRACTS ...,NaN,1925.0,Out of State,43,NaN,No,Yes,No,No,No,NaN,NaN,No,NaN,NaN,75226,Other - PD-269,District 2,2022-04-01,0.114955,NaN,524.0,Other,No,Other


In [16]:
vacant_lots_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33941 entries, 0 to 39644
Data columns (total 31 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   Parcel ID                               33941 non-null  object        
 1   Adddress                                33941 non-null  object        
 2   Owner                                   33941 non-null  object        
 3   Is Vacant Property                      33941 non-null  object        
 4   Property Type                           32890 non-null  object        
 5   Current Use                             33941 non-null  object        
 6   Year Built                              8564 non-null   float64       
 7   Average Year Built By Council District  33931 non-null  float64       
 8   Owner Location                          33941 non-null  object        
 9   Owner Property Count                    33941 non-null 

In [17]:
vacant_lots_df.isnull().sum()

Parcel ID                                     0
Adddress                                      0
Owner                                         0
Is Vacant Property                            0
Property Type                              1051
Current Use                                   0
Year Built                                25377
Average Year Built By Council District       10
Owner Location                                0
Owner Property Count                          0
Public Owner                              31524
City or County Owned                          0
Owner is Business                             0
Owner Occupied                                0
Suspected Heir Property                       0
Bank Owned Property                           0
Taxes Delinquent Since                    29571
Delinquent Taxes                          29571
Is Tax Delinquent                             0
Code Liens - Count                        28316
Code Liens - Amount                     

In [18]:
vacant_lots_df.columns

Index(['Parcel ID', 'Adddress', 'Owner', 'Is Vacant Property', 'Property Type',
       'Current Use', 'Year Built', 'Average Year Built By Council District',
       'Owner Location', 'Owner Property Count', 'Public Owner',
       'City or County Owned', 'Owner is Business', 'Owner Occupied',
       'Suspected Heir Property', 'Bank Owned Property',
       'Taxes Delinquent Since', 'Delinquent Taxes', 'Is Tax Delinquent',
       'Code Liens - Count', 'Code Liens - Amount', 'Zip Code', 'Zoning',
       'Council District', 'Vacant Since', 'Land Size', 'Property Age Years',
       'Vacancy Days', 'Zoning Category 3', 'Owner is Church',
       'Zoning Category'],
      dtype='object')

In [19]:
#Sets results for dict
results = {}

In [20]:
#1.How many vacant lots are in each council district?

#Groups Council distrcits column and 
vacant_lots_counts_by_district  = land_df.groupby('Council District').size()


In [21]:
#dictonary that starts the result values
results["How many vacant lots are in each council district?"] = {
    'question_number': 1,
    'value': vacant_lots_counts_by_district
}

results["How many vacant lots are in each council district?"]


{'question_number': 1,
 'value': Council District
 District 1     1159
 District 10     296
 District 11     311
 District 12      99
 District 13     392
 District 14     967
 District 2     2150
 District 3     1144
 District 4     3435
 District 5     1459
 District 6     3745
 District 7     4426
 District 8     2310
 District 9      195
 dtype: int64}

In [22]:
#2 How long have the vacant lots in each council district been vacant? 
vacant_lots_stats = land_df.groupby('Council District')['Vacancy Days'].agg(['min','median', 'mean', 'max'])

# Rename the columns for clarity
vacant_lots_stats.columns = ['Min Days', 'Median Days', 'Mean Days', 'Max Days']


In [23]:

results["How long have the vacant lots in each council district been vacant?"] = {
    'question_number': 2,
    'value': vacant_lots_stats
}


results["How long have the vacant lots in each council district been vacant?"]

{'question_number': 2,
 'value':                   Min Days  Median Days   Mean Days  Max Days
 Council District                                             
 District 1            98.0        524.0  501.861087     524.0
 District 10           98.0        524.0  506.871622     524.0
 District 11           98.0        524.0  502.186495     524.0
 District 12           98.0        524.0  516.858586     524.0
 District 13           98.0        524.0  515.683673     524.0
 District 14           71.0        524.0  510.965874     524.0
 District 2            98.0        524.0  512.792558     524.0
 District 3            98.0        524.0  520.027122     524.0
 District 4            98.0        524.0  513.671325     524.0
 District 5            98.0        524.0  518.831391     524.0
 District 6            98.0        524.0  511.835514     524.0
 District 7            98.0        524.0  517.581563     524.0
 District 8            98.0        524.0  517.618615     524.0
 District 9            

In [24]:
# 3 How many are zoned single family residential, commercial and multifamily? <-- rows are council District; columns are counts within each of the 3 zoning types

# Group by 'Council District' and 'Zoning Category', then count occurrences
#zoning_category_counts = vacant_lots_df.groupby(['Council District', 'Zoning Category']).size().unstack(fill_value=0)


In [25]:

'''results[" How many are zoned single family residential, commercial and multifamily? <-- rows are council District; columns are counts within each of the 3 zoning types"] = {
    'question_number': 3,
    'value': zoning_category_counts
}

results'''

'results[" How many are zoned single family residential, commercial and multifamily? <-- rows are council District; columns are counts within each of the 3 zoning types"] = {\n    \'question_number\': 3,\n    \'value\': zoning_category_counts\n}\n\nresults'

In [26]:
#3 How many vacant lots in each council district are zoned single family residential, commercial and multifamily 

# Group by 'Council District' and 'Zoning Category', then count occurrences
zoning_category_counts_3 = land_df.groupby(['Council District', 'Zoning Category 3']).size().unstack(fill_value=0)


In [27]:

results["How many are zoned single family residential, commercial and multifamily?"] = {
    'question_number': 3,
    'value': zoning_category_counts_3
}


results["How many are zoned single family residential, commercial and multifamily?"]

{'question_number': 3,
 'value': Zoning Category 3  Commercial  Multifamily  Other  Single Family
 Council District                                                
 District 1                  3           65    832            213
 District 10                 8           54    158             34
 District 11                 0           15    228             68
 District 12                 0           18     51             30
 District 13                 1           65    164            136
 District 14                 1           92    608              5
 District 2                182          223   1267            111
 District 3                 22           42    435            620
 District 4                 93          254   1284           1802
 District 5                 12           41    582            824
 District 6                112           91   1594           1257
 District 7                149          236    760            370
 District 8                129           71 

In [28]:
# 4. How many vacant structures are in each council district?

# Groups Council District by structure and vacant property

# Grouping by 'Council District' and counting occurrences
vacant_structure_count = structure_df.groupby('Council District').size()

In [29]:

results["How many vacant structures are in each council district?"] = {
    'question_number': 4,
    'value': vacant_structure_count
}

results["How many vacant structures are in each council district?"]

{'question_number': 4,
 'value': Council District
 District 1      574
 District 10     294
 District 11     280
 District 12      68
 District 13     963
 District 14     716
 District 2     1232
 District 3      378
 District 4     1728
 District 5      588
 District 6     1477
 District 7     1170
 District 8      653
 District 9      671
 dtype: int64}

In [30]:
#5 How long have the vacant structures in each council district been vacant? (see question 2)
vacant_structure_count_days = structure_df.groupby('Council District')['Vacancy Days'].agg(['min','median', 'mean', 'max'])

# Rename the columns for clarity
vacant_structure_count_days.columns = ['Min Days', 'Median Days', 'Mean Days', 'Max Days']


In [31]:

results["How long have the vacant structures in each council district been vacant? (see question 2)"] = {
    'question_number': 5,
    'value': vacant_structure_count_days
}

results["How long have the vacant structures in each council district been vacant? (see question 2)"]

{'question_number': 5,
 'value':                   Min Days  Median Days   Mean Days  Max Days
 Council District                                             
 District 1            98.0        524.0  454.083624     524.0
 District 10           98.0        524.0  363.874150     524.0
 District 11           98.0        524.0  393.510714     524.0
 District 12           98.0        243.0  295.808824     524.0
 District 13           98.0        524.0  424.689512     524.0
 District 14           98.0        524.0  394.997207     524.0
 District 2            98.0        524.0  424.135552     524.0
 District 3            98.0        524.0  447.264550     524.0
 District 4            98.0        524.0  437.689236     524.0
 District 5            98.0        524.0  461.197279     524.0
 District 6            98.0        524.0  446.817874     524.0
 District 7            98.0        524.0  439.519658     524.0
 District 8            98.0        524.0  469.808576     524.0
 District 9            

In [32]:

#6 How many vacant structures in each council district are zoned single family residential, commercial and multifamily? (see question 3)

zoning_category_counts_structure = structure_df.groupby(['Council District', 'Zoning Category 3']).size().unstack(fill_value=0)


In [33]:

results["How many vacant structures in each council district are zoned single family residential, commercial and multifamily? (see question 3)"] = {
    'question_number': 6,
    'value': zoning_category_counts_structure
}

results["How many vacant structures in each council district are zoned single family residential, commercial and multifamily? (see question 3)"] 


{'question_number': 6,
 'value': Zoning Category 3  Commercial  Multifamily  Other  Single Family
 Council District                                                
 District 1                  4           30    281            228
 District 10                 6           18    140             45
 District 11                 0           27     95            158
 District 12                 0           15     17             36
 District 13                 0           35    237            671
 District 14                 1           89    277             24
 District 2                 47          164    437             48
 District 3                  5           12     80            279
 District 4                 12           75    272           1368
 District 5                  6           31     92            459
 District 6                 16           59    518            615
 District 7                 26           81     95            224
 District 8                 11           13 

In [34]:
#7.How many vacant lots are in each zip code?

# Groups the zip code and if property is vacant and produces the count 
vacant_lots_counts_by_zip = land_df.groupby('Zip Code').size()

# Convert index (zip codes) to integers to remove the .0 decimal part
vacant_lots_counts_by_zip.index = vacant_lots_counts_by_zip.index.astype(int)

In [35]:

results["How many vacant lots are in each zip code?"] = {
    'question_number': 7,
    'value': vacant_lots_counts_by_zip
}

results["How many vacant lots are in each zip code?"]

{'question_number': 7,
 'value': Zip Code
 75006       4
 75019      19
 75041       1
 75043       5
 75050       1
 75051       5
 75060       1
 75062       1
 75063       2
 75080       9
 75081       2
 75141       4
 75149       9
 75150       4
 75159       3
 75182       1
 75201     324
 75202     121
 75203    2054
 75204     443
 75205      35
 75206     225
 75207     206
 75208     796
 75209     106
 75210     976
 75211     693
 75212    2336
 75214     113
 75215    2687
 75216    1845
 75217    1640
 75218      83
 75219     311
 75220     355
 75223     796
 75224     219
 75225      90
 75226     437
 75227     499
 75228     167
 75229     316
 75230     155
 75231     137
 75232     260
 75233     153
 75234      18
 75235     289
 75236     222
 75237     248
 75238      69
 75240      70
 75241    1427
 75243     210
 75244      12
 75246     124
 75247     242
 75248     120
 75249      58
 75251       7
 75252      18
 75253     269
 75254      44
 75270       

In [36]:
#8 How long have the vacant lots in each zip code been vacant?
vacant_zips_count_days = land_df.groupby('Zip Code')['Vacancy Days'].agg(['min','median', 'mean', 'max'])

# Convert index (zip codes) to integers to remove the .0 decimal part
vacant_zips_count_days.index = vacant_zips_count_days.index.astype(int)

# Rename the columns for clarity
vacant_zips_count_days.columns = ['Min Days', 'Median Days', 'Mean Days', 'Max Days']

In [37]:

results["How long have the vacant lots in each zip code been vacant?"] = {
    'question_number': 8,
    'value': vacant_zips_count_days
}

results["How long have the vacant lots in each zip code been vacant?"] 

{'question_number': 8,
 'value':           Min Days  Median Days   Mean Days  Max Days
 Zip Code                                             
 75006        524.0        524.0  524.000000     524.0
 75019         98.0        524.0  479.157895     524.0
 75041        524.0        524.0  524.000000     524.0
 75043        524.0        524.0  524.000000     524.0
 75050        524.0        524.0  524.000000     524.0
 75051        524.0        524.0  524.000000     524.0
 75060        524.0        524.0  524.000000     524.0
 75062        524.0        524.0  524.000000     524.0
 75063        524.0        524.0  524.000000     524.0
 75080        524.0        524.0  524.000000     524.0
 75081        524.0        524.0  524.000000     524.0
 75141        524.0        524.0  524.000000     524.0
 75149        243.0        524.0  492.777778     524.0
 75150        371.0        524.0  485.750000     524.0
 75159        524.0        524.0  524.000000     524.0
 75182        524.0        524.0 

In [38]:
#9 How many vacant lots in each zip code are zoned single family residential, commercial and multifamily?


zoning_category_counts_zip = land_df.groupby(['Zip Code', 'Zoning Category 3']).size().unstack(fill_value=0)

# Convert index (zip codes) to integers to remove the .0 decimal part
zoning_category_counts_zip.index = zoning_category_counts_zip.index.astype(int)

In [39]:

results["How many vacant lots in each zip code are zoned single family residential, commercial and multifamily?"] = {
    'question_number': 9,
    'value': zoning_category_counts_zip
}

results["How many vacant lots in each zip code are zoned single family residential, commercial and multifamily?"]

{'question_number': 9,
 'value': Zoning Category 3  Commercial  Multifamily  Other  Single Family
 Zip Code                                                        
 75006                       0            0      4              0
 75019                       0            0      3              0
 75043                       0            0      4              0
 75051                       0            0      1              4
 75060                       0            0      1              0
 75062                       0            0      1              0
 75080                       0            0      9              0
 75081                       0            1      1              0
 75141                       0            0      4              0
 75149                       0            0      9              0
 75150                       0            0      2              0
 75159                       0            0      1              2
 75182                       0            0 

In [40]:
#10 How many vacant structures are in each zip code?

# Groups the zip code and if property is vacant and produces the count 
vacant_structure_counts_by_zip = structure_df.groupby('Zip Code').size()

# Convert index (zip codes) to integers to remove the .0 decimal part
vacant_structure_counts_by_zip.index = vacant_structure_counts_by_zip.index.astype(int)

In [41]:

results["How many vacant structures are in each zip code?"] = {
    'question_number': 10,
    'value': vacant_structure_counts_by_zip
}

results["How many vacant structures are in each zip code?"]

{'question_number': 10,
 'value': Zip Code
 75006       1
 75042       1
 75063       1
 75080       1
 75081       1
 75115       1
 75150       1
 75201     129
 75202      37
 75203     394
 75204     204
 75205      63
 75206     255
 75207     167
 75208     317
 75209     316
 75210     241
 75211     304
 75212     676
 75214     270
 75215     521
 75216    1403
 75217     508
 75218     331
 75219     246
 75220     328
 75223     175
 75224     119
 75225     142
 75226      66
 75227     392
 75228     663
 75229     457
 75230     215
 75231     126
 75232     180
 75233      52
 75234      19
 75235     184
 75236      46
 75237      44
 75238     103
 75240      65
 75241     449
 75243     158
 75244      63
 75246      33
 75247      83
 75248     117
 75249      18
 75251       9
 75252       2
 75253      71
 75254      24
 dtype: int64}

In [42]:
#11 How long have the vacant structures in each zip code been vacant? (see question 8)
vacant_zips_structure_count_days = structure_df.groupby('Zip Code')['Vacancy Days'].agg(['min','median', 'mean', 'max'])

# Convert index (zip codes) to integers to remove the .0 decimal part
vacant_zips_structure_count_days.index = vacant_zips_structure_count_days.index.astype(int)

# Rename the columns for clarity
vacant_zips_structure_count_days.columns = ['Min Days', 'Median Days', 'Mean Days', 'Max Days']

In [43]:

results["How long have the vacant structures in each zip code been vacant? (see question 8)"] = {
    'question_number': 11,
    'value': vacant_zips_structure_count_days
}

results["How long have the vacant structures in each zip code been vacant? (see question 8)"]

{'question_number': 11,
 'value':           Min Days  Median Days   Mean Days  Max Days
 Zip Code                                             
 75006        524.0        524.0  524.000000     524.0
 75042         98.0         98.0   98.000000      98.0
 75063        524.0        524.0  524.000000     524.0
 75080         98.0         98.0   98.000000      98.0
 75081        524.0        524.0  524.000000     524.0
 75115        336.0        336.0  336.000000     336.0
 75150        371.0        371.0  371.000000     371.0
 75201         98.0        243.0  321.085271     524.0
 75202         98.0        190.0  306.918919     524.0
 75203         98.0        524.0  444.758883     524.0
 75204         98.0        524.0  438.137255     524.0
 75205         98.0        524.0  435.968254     524.0
 75206         98.0        524.0  430.901961     524.0
 75207         98.0        524.0  423.233533     524.0
 75208         98.0        524.0  450.611987     524.0
 75209         98.0        524.0

In [44]:
#12 How many vacant structures in each zip code are zoned single family residential, commercial and multifamily? (see question 9)

zoning_category_counts_structure_zip = structure_df.groupby(['Zip Code', 'Zoning Category 3']).size().unstack(fill_value=0)

# Convert index (zip codes) to integers to remove the .0 decimal part
zoning_category_counts_structure_zip.index = zoning_category_counts_structure_zip.index.astype(int)

In [45]:

results["How many vacant structures in each zip code are zoned single family residential, commercial and multifamily? (see question 9)"] = {
    'question_number': 12,
    'value': zoning_category_counts_structure_zip
}

results["How many vacant structures in each zip code are zoned single family residential, commercial and multifamily? (see question 9)"]


{'question_number': 12,
 'value': Zoning Category 3  Commercial  Multifamily  Other  Single Family
 Zip Code                                                        
 75006                       0            0      1              0
 75042                       0            0      1              0
 75080                       0            0      0              1
 75081                       0            0      1              0
 75115                       0            0      1              0
 75150                       0            0      1              0
 75201                       0            0     80              0
 75202                       0            0     37              0
 75203                       1           28    207            157
 75204                       4           95     87              1
 75205                       0            1      2              0
 75206                       1           92    115             19
 75207                       0            0

In [46]:
#13 What is the total amount of back taxes owed on vacant land in each City Council District?

# Grouping and calculating the total amount of back taxes owed on vacant land by City Council District
#Remove vacant prop yes filter
back_taxes_total_by_district = land_df.groupby('Council District')['Delinquent Taxes'].sum()

In [47]:

results["What is the total amount of back taxes owed on vacant land in each City Council District?"] = {
    'question_number': 13,
    'value': back_taxes_total_by_district
}

results["What is the total amount of back taxes owed on vacant land in each City Council District?"] 

{'question_number': 13,
 'value': Council District
 District 1      686.88
 District 10      73.07
 District 11      36.63
 District 12      12.38
 District 13    3349.69
 District 14    1166.07
 District 2     2729.34
 District 3     1047.80
 District 4     2489.74
 District 5      737.64
 District 6     2722.43
 District 7     2072.02
 District 8     1661.75
 District 9       34.73
 Name: Delinquent Taxes, dtype: float64}

In [48]:
#14 What is the total amount of back taxes owed on vacant land in each zip code?

# Grouping and calculating the total amount of back taxes owed on vacant land by Zip Code
back_taxes_total_by_zip = land_df.groupby('Zip Code')['Delinquent Taxes'].sum()

# Convert index (zip codes) to integers to remove the .0 decimal part
back_taxes_total_by_zip.index = back_taxes_total_by_zip.index.astype(int)


In [49]:

results["What is the total amount of back taxes owed on vacant land in each zip code?"] = {
    'question_number': 14,
    'value': back_taxes_total_by_zip
}

results["What is the total amount of back taxes owed on vacant land in each zip code?"] 

{'question_number': 14,
 'value': Zip Code
 75006       0.00
 75019       0.05
 75041       0.00
 75043       0.00
 75050       0.00
 75051       3.82
 75060       0.00
 75062       0.00
 75063       0.00
 75080       0.01
 75081       0.16
 75141       0.00
 75149       0.00
 75150       0.02
 75159       0.00
 75182       0.00
 75201     670.36
 75202      47.63
 75203    1061.66
 75204     511.79
 75205      12.85
 75206     114.97
 75207      52.91
 75208     715.16
 75209     102.16
 75210     350.29
 75211     397.62
 75212    1924.93
 75214      70.88
 75215    1590.29
 75216    1466.31
 75217     945.77
 75218       0.57
 75219     410.38
 75220    3033.48
 75223     463.30
 75224     151.16
 75225      66.34
 75226    1114.80
 75227     343.15
 75228      73.90
 75229      43.41
 75230       1.52
 75231     554.58
 75232     349.80
 75233     105.12
 75234       0.00
 75235     179.47
 75236     143.34
 75237     536.98
 75238       9.92
 75240      30.22
 75241     980.39
 75

In [50]:
#15 What is the total amount of back taxes owed on vacant structures in each City Council District?

# Grouping and calculating the total amount of back taxes owed on vacant land by City Council District
back_taxes_total_by_district_by_structure = structure_df.groupby('Council District')['Delinquent Taxes'].sum()


In [51]:

results["What is the total amount of back taxes owed on vacant structures in each City Council District?"] = {
    'question_number': 15,
    'value': back_taxes_total_by_district_by_structure
}

results["What is the total amount of back taxes owed on vacant structures in each City Council District?"]

{'question_number': 15,
 'value': Council District
 District 1      595.25
 District 10     157.51
 District 11     675.19
 District 12     105.98
 District 13    3978.19
 District 14    2212.51
 District 2     1108.95
 District 3      443.67
 District 4     1804.12
 District 5      677.84
 District 6     2080.38
 District 7      836.17
 District 8      540.02
 District 9      866.64
 Name: Delinquent Taxes, dtype: float64}

In [52]:
#16 What is the total amount of back taxes owed on vacant structures in each zip code?

# Grouping and calculating the total amount of back taxes owed on vacant land by Zip Code
back_taxes_total_by_zip_by_structure = structure_df.groupby('Zip Code')['Delinquent Taxes'].sum()

# Convert index (zip codes) to integers to remove the .0 decimal part
back_taxes_total_by_zip_by_structure.index = back_taxes_total_by_zip_by_structure.index.astype(int)


In [53]:

results["What is the total amount of back taxes owed on vacant structures in each zip code?"] = {
    'question_number': 16,
    'value': back_taxes_total_by_zip_by_structure
}

results["What is the total amount of back taxes owed on vacant structures in each zip code?"] 

{'question_number': 16,
 'value': Zip Code
 75006       0.00
 75042       0.00
 75063       0.00
 75080       0.00
 75081       0.00
 75115      48.01
 75150       0.01
 75201    1042.72
 75202       0.00
 75203     399.33
 75204     299.72
 75205     179.92
 75206     377.25
 75207     206.54
 75208     524.42
 75209     940.60
 75210     199.91
 75211     256.09
 75212     506.37
 75214     456.50
 75215     293.40
 75216    1406.63
 75217     543.47
 75218     595.17
 75219     238.44
 75220    1708.26
 75223     468.43
 75224      60.67
 75225     471.83
 75226       5.69
 75227     491.98
 75228     419.99
 75229    1404.10
 75230     658.43
 75231     495.71
 75232     166.81
 75233      88.56
 75234       0.00
 75235      71.04
 75236       7.20
 75237      56.99
 75238      59.41
 75240      27.56
 75241     380.76
 75243     127.37
 75244      71.40
 75246      81.20
 75247       6.56
 75248     116.08
 75249      15.40
 75251       0.00
 75252       0.00
 75253      51.94
 75

In [54]:
#17 What is the average age of residential properties in each council district?

#avg_age__prop_District=vacant_lots_df.groupby('Council District')['Property Age Years'].mean()
# Calculate the average age of residential properties

residential_avg_age_prop_District = residential_properties.groupby('Council District')['Property Age Years'].mean()

In [55]:

results["What is the average age of residential properties in each council district?"] = {
    'question_number': 17,
    'value': residential_avg_age_prop_District
}

results["What is the average age of residential properties in each council district?"]

{'question_number': 17,
 'value': Council District
 District 1     65.523810
 District 10    50.153846
 District 11    37.125000
 District 12    42.511111
 District 13    27.321053
 District 14    20.376623
 District 2     27.183432
 District 3     41.467181
 District 4     54.242220
 District 5     53.925258
 District 6     23.497984
 District 7     39.169811
 District 8     29.733591
 District 9     55.355932
 Name: Property Age Years, dtype: float64}

In [56]:
#18 What is the average age of residential properties in each zip code?
avg_age__prop_zip = residential_properties.groupby('Zip Code')['Property Age Years'].mean()
#Fills na values with 0
avg_age__prop_zip=avg_age__prop_zip.fillna(0)

#converts avg age to int data type to remve decimals
avg_age__prop_zip.index=avg_age__prop_zip.index.astype(int)


In [57]:

results["What is the average age of residential properties in each zip code?"] = {
    'question_number': 18,
    'value': avg_age__prop_zip
}

results["What is the average age of residential properties in each zip code?"]

{'question_number': 18,
 'value': Zip Code
 75051     0.000000
 75080     0.000000
 75081     0.000000
 75159     0.000000
 75203    51.617450
 75204    13.553846
 75205     0.000000
 75206    16.200000
 75208    60.619048
 75209    25.297619
 75210    52.200000
 75211    59.631868
 75212    18.623229
 75214    40.268293
 75215     0.000000
 75216    51.042927
 75217    37.030848
 75218    64.270270
 75220    24.234973
 75223    53.152542
 75224    66.818182
 75225    22.158537
 75226     1.285714
 75227    52.782178
 75228    57.846154
 75229    36.723502
 75230    18.044444
 75231    35.866667
 75232    47.106383
 75233    25.971429
 75234    61.250000
 75235    21.952381
 75236    25.157895
 75237     8.555556
 75238    38.500000
 75240    51.777778
 75241    42.383929
 75243    50.782609
 75244    36.840000
 75246     1.666667
 75248    45.075949
 75249    44.250000
 75252     0.000000
 75253    10.555556
 75254    49.153846
 Name: Property Age Years, dtype: float64}

In [58]:
#19 What is the average size of residential properties in each council district?


# Group by council district and calculate the average property size
average_size_by_district = residential_properties.groupby('Council District')['Land Size'].mean()

In [59]:

results["What is the average size of residential properties in each council district?"] = {
    'question_number': 19,
    'value': average_size_by_district
}

results["What is the average size of residential properties in each council district?"]

{'question_number': 19,
 'value': Council District
 District 1     0.334292
 District 10    1.778136
 District 11    1.520976
 District 12    1.611201
 District 13    1.063923
 District 14    0.296096
 District 2     0.278586
 District 3     1.058363
 District 4     0.317044
 District 5     0.737821
 District 6     0.220295
 District 7     1.129251
 District 8     1.484604
 District 9     0.699083
 Name: Land Size, dtype: float64}

In [60]:
#20 What is the average size of residential properties in each zip code?

# Group by council district and calculate the average property size
average_size_by_zip = residential_properties.groupby('Zip Code')['Land Size'].mean()

# Convert index (zip codes) to integers to remove the .0 decimal part
average_size_by_zip.index = average_size_by_zip.index.astype(int)


In [61]:

results["What is the average size of residential properties in each zip code?"] = {
    'question_number': 20,
    'value': average_size_by_zip
}

results["What is the average size of residential properties in each zip code?"]

{'question_number': 20,
 'value': Zip Code
 75051     4.935997
 75080    21.995701
 75081     0.107777
 75159     0.230563
 75203     0.229260
 75204     0.184217
 75205     0.155813
 75206     0.314645
 75208     0.274553
 75209     0.774986
 75210     1.280558
 75211     0.661935
 75212     0.195813
 75214     0.477058
 75215    35.864064
 75216     0.351181
 75217     1.084001
 75218     0.474402
 75220     0.501091
 75223     0.243278
 75224     0.628213
 75225     0.533070
 75226     0.205823
 75227     0.945864
 75228     1.463396
 75229     0.715911
 75230     0.590945
 75231     5.730431
 75232     0.959853
 75233     0.712694
 75234     0.197529
 75235     0.194761
 75236     2.566795
 75237     1.577490
 75238     0.402267
 75240     1.337079
 75241     1.191478
 75243     1.988550
 75244     0.463680
 75246     0.260339
 75248     1.412633
 75249     0.518870
 75252     0.846749
 75253     1.758012
 75254     5.563971
 Name: Land Size, dtype: float64}

In [62]:
#21 What is the average age of commercial properties in each council district?

# Group by council district and calculate the average property size by commercial prop
average_age_by_district_commercial = commercial_properties.groupby('Council District')['Property Age Years'].mean()


In [63]:

results["What is the average age of commercial properties in each council district?"] = {
    'question_number': 21,
    'value': average_age_by_district_commercial
}

results["What is the average age of commercial properties in each council district?"]

{'question_number': 21,
 'value': Council District
 District 1     56.500000
 District 10    39.800000
 District 13          NaN
 District 14    97.000000
 District 2     59.344828
 District 3      9.000000
 District 4     72.571429
 District 5     37.333333
 District 6     55.400000
 District 7     67.157895
 District 8     43.000000
 District 9     65.000000
 Name: Property Age Years, dtype: float64}

In [64]:
#22 What is the average age of commercial properties in each zip code?

average_age_by_zip_commercial = commercial_properties.groupby('Zip Code')['Property Age Years'].mean()

# Convert index (zip codes) to integers to remove the .0 decimal part
average_age_by_zip_commercial.index = average_age_by_zip_commercial.index.astype(int)

In [65]:
results["What is the average age of commercial properties in each zip code?"] = {
    'question_number': 22,
    'value': average_age_by_zip_commercial
}

results["What is the average age of commercial properties in each zip code?"]

{'question_number': 22,
 'value': Zip Code
 75203    88.000000
 75204    75.750000
 75206    97.000000
 75207          NaN
 75208    56.000000
 75209          NaN
 75210          NaN
 75211    13.500000
 75212    75.000000
 75215          NaN
 75216    63.500000
 75217    38.750000
 75218    65.000000
 75220    28.000000
 75223    59.882353
 75224          NaN
 75226    74.000000
 75227          NaN
 75228    61.000000
 75229          NaN
 75235    68.500000
 75236     0.000000
 75237          NaN
 75238    40.000000
 75241          NaN
 75243    39.000000
 75246          NaN
 75247    63.000000
 75253          NaN
 Name: Property Age Years, dtype: float64}

In [66]:
#23 What is the total amount of city liens owed on vacant land in each City Council District?

# Grouping and calculating the total amount of city liens owed on vacant land by City Council District
city_liens_total_by_district = land_df.groupby('Council District')['Code Liens - Amount'].sum()


In [67]:

results["What is the total amount of city liens owed on vacant land in each City Council District?"] = {
    'question_number': 23,
    'value': city_liens_total_by_district
}

results["What is the total amount of city liens owed on vacant land in each City Council District?"]

{'question_number': 23,
 'value': Council District
 District 1      103530.19
 District 10       5504.10
 District 11       9571.56
 District 12          0.00
 District 13       3531.70
 District 14      23623.42
 District 2      271934.23
 District 3      153084.99
 District 4     2370252.74
 District 5      548893.25
 District 6      518491.43
 District 7     5400356.45
 District 8      746219.63
 District 9       10579.79
 Name: Code Liens - Amount, dtype: float64}

In [68]:
#24 What is the total amount of city liens owed on vacant land in each zip code?

# Grouping and calculating the total amount of city liens owed on vacant land by City Council District
city_liens_total_by_zip = land_df.groupby('Zip Code')['Code Liens - Amount'].sum()

# Convert index (zip codes) to integers to remove the .0 decimal part
city_liens_total_by_zip.index = city_liens_total_by_zip.index.astype(int)


In [69]:

results["What is the total amount of city liens owed on vacant land in each zip code?"] = {
    'question_number': 24,
    'value': city_liens_total_by_zip
}

results["What is the total amount of city liens owed on vacant land in each zip code?"]

{'question_number': 24,
 'value': Zip Code
 75006          0.00
 75019          0.00
 75041          0.00
 75043          0.00
 75050          0.00
 75051        529.14
 75060          0.00
 75062          0.00
 75063          0.00
 75080          0.00
 75081          0.00
 75141          0.00
 75149        171.41
 75150          0.00
 75159          0.00
 75182          0.00
 75201          0.00
 75202          0.00
 75203     883999.05
 75204      61924.96
 75205          0.00
 75206       6015.15
 75207       3035.14
 75208      42396.10
 75209       6726.82
 75210    1441588.02
 75211      80063.23
 75212     449338.43
 75214       7111.97
 75215    3443115.72
 75216    1637717.43
 75217     581928.74
 75218       4161.43
 75219      11045.10
 75220      34000.09
 75223     380387.86
 75224      72192.63
 75225        239.61
 75226      70241.85
 75227      85960.74
 75228      13774.09
 75229       3692.99
 75230       2692.20
 75231          0.00
 75232     109812.48
 75233      

In [70]:
#25 What is the total amount of city liens owed on vacant structures in each City Council District?

# Grouping and calculating the total amount of city liens owed on vacant structures by City Council District
city_liens_total_by_district_by_structure = structure_df.groupby('Council District')['Code Liens - Amount'].sum()


In [71]:

results["What is the total amount of city liens owed on vacant structures in each City Council District?"] = {
    'question_number': 25,
    'value': city_liens_total_by_district_by_structure
}

results["What is the total amount of city liens owed on vacant structures in each City Council District?"]

{'question_number': 25,
 'value': Council District
 District 1      50196.85
 District 10      2680.68
 District 11     32389.45
 District 12     49869.82
 District 13      8414.82
 District 14     15862.81
 District 2      67556.76
 District 3      81955.43
 District 4     825379.21
 District 5     115061.87
 District 6     133324.19
 District 7     484660.67
 District 8     174351.60
 District 9      38464.13
 Name: Code Liens - Amount, dtype: float64}

In [72]:
#26 How many properties are in the name of heirship in the City of Dallas?

#Use suspected heir prop col and combine with public owner filter
# Filter the DataFrame for properties in the City of Dallas with 'Suspected Heir Property' == 'Yes'
#Would get zero combined with City of Dallas filter
dallas_heirship_properties = (vacant_lots_df['Suspected Heir Property'] == 'Yes').sum()



In [73]:

results["How many properties are in the name of heirship in the City of Dallas?"] = {
    'question_number': 26,
    'value': dallas_heirship_properties
}

results["How many properties are in the name of heirship in the City of Dallas?"] 

{'question_number': 26, 'value': 248}

In [74]:
#27 How many properties in the name of heirship in the City of Dallas are vacant in each zip code?

#apply Suspected heir to Yes and group by zip code
dallas_heirship_zip = vacant_lots_df[vacant_lots_df['Suspected Heir Property'] == 'Yes'].groupby('Zip Code').size()

# Convert index (zip codes) to integers to remove the .0 decimal part
dallas_heirship_zip.index = dallas_heirship_zip.index.astype(int)


In [75]:

results["How many properties in the name of heirship in the City of Dallas are vacant in each zip code?"] = {
    'question_number': 27,
    'value': dallas_heirship_zip
}

results["How many properties in the name of heirship in the City of Dallas are vacant in each zip code?"] 

{'question_number': 27,
 'value': Zip Code
 75201     4
 75203    21
 75206     2
 75207     1
 75208     3
 75209     1
 75210    13
 75211     3
 75212    21
 75214     3
 75215    35
 75216    48
 75217    11
 75218     3
 75220     2
 75223     5
 75224     1
 75225     3
 75226     4
 75227     9
 75228     9
 75230     1
 75231     2
 75232     4
 75233     1
 75235     1
 75237     4
 75238     1
 75241    24
 75243     1
 75246     1
 75248     1
 75253     5
 dtype: int64}

In [76]:
#28 How many properties in the name of heirship in the City of Dallas are vacant in each council district?

#apply Suspected heir to Yes and group by council distrcit
dallas_heirship_District = vacant_lots_df[vacant_lots_df['Suspected Heir Property'] == 'Yes'].groupby('Council District').size()


In [77]:

results["How many properties in the name of heirship in the City of Dallas are vacant in each council district?"] = {
    'question_number': 28,
    'value': dallas_heirship_District
}

results["How many properties in the name of heirship in the City of Dallas are vacant in each council district?"] 

{'question_number': 28,
 'value': Council District
 District 1      4
 District 10     4
 District 11     2
 District 13     4
 District 14     8
 District 2     12
 District 3     10
 District 4     58
 District 5     16
 District 6     24
 District 7     69
 District 8     31
 District 9      6
 dtype: int64}

In [78]:
#29 How many private owners have five or more vacant properties in the City of Dallas?
owner_vacant_property_counts = vacant_lots_df[
    (vacant_lots_df['Owner'] == 'Private Owner') &
    (vacant_lots_df['Owner Property Count'] >= 5)
]

owner_vacant_property_counts_view = owner_vacant_property_counts.shape[0]


In [79]:

results["How many private owners have five or more vacant properties in the City of Dallas?"] = {
    'question_number': 29,
    'value': owner_vacant_property_counts_view
}

results["How many private owners have five or more vacant properties in the City of Dallas?"] 

{'question_number': 29, 'value': 6702}

In [80]:
#30 How many private owners with five or more vacant properties in the City of Dallas are in each zip code?

private_5_zip = owner_vacant_property_counts.groupby('Zip Code').size()

# Convert index (zip codes) to integers to remove the .0 decimal part
private_5_zip.index = private_5_zip.index.astype(int)


In [81]:

results["How many private owners with five or more vacant properties in the City of Dallas are in each zip code?"] = {
    'question_number': 30,
    'value': private_5_zip
}

results["How many private owners with five or more vacant properties in the City of Dallas are in each zip code?"]

{'question_number': 30,
 'value': Zip Code
 75006      1
 75019      8
 75043      1
 75050      1
 75051      1
 75063      1
 75080      1
 75081      1
 75149      1
 75182      1
 75201     55
 75202     21
 75203    562
 75204     86
 75205     12
 75206     70
 75207     49
 75208    217
 75209     57
 75210    231
 75211    305
 75212    900
 75214     33
 75215    653
 75216    663
 75217    412
 75218     60
 75219     63
 75220    119
 75223    216
 75224     61
 75225     66
 75226    163
 75227    172
 75228     96
 75229    114
 75230    109
 75231     36
 75232     76
 75233     51
 75234      4
 75235    145
 75236     50
 75237     96
 75238     17
 75240      9
 75241    315
 75243     91
 75244      8
 75246     57
 75247     62
 75248     24
 75249     12
 75251      2
 75252      8
 75253     53
 75254      4
 dtype: int64}

In [82]:
#31 How many private owners with five or more vacant properties in the City of Dallas are in each council district?

private_5_district=owner_vacant_property_counts.groupby('Council District').size()

In [83]:

results["How many private owners with five or more vacant properties in the City of Dallas are in each council district?"] = {
    'question_number': 31,
    'value': private_5_district
}

results["How many private owners with five or more vacant properties in the City of Dallas are in each council district?"]

{'question_number': 31,
 'value': Council District
 District 1      416
 District 10     121
 District 11     121
 District 12      30
 District 13     168
 District 14     251
 District 2      607
 District 3      397
 District 4     1026
 District 5      409
 District 6     1375
 District 7     1175
 District 8      500
 District 9      103
 dtype: int64}

In [84]:
#32 How many churches own vacant properties in the City of Dallas?

church_owners = vacant_lots_df[vacant_lots_df['Owner is Church'] == 'Yes']

num_church_properties = church_owners.shape[0]



In [85]:

results["How many churches own vacant properties in the City of Dallas?"] = {
    'question_number': 32,
    'value': num_church_properties
}

results["How many churches own vacant properties in the City of Dallas?"] 

{'question_number': 32, 'value': 35}

In [86]:
#33 How many churches owned vacant properties in the City of Dallas are in each zip code?

church_prop_zip = vacant_lots_df[vacant_lots_df['Owner is Church'] == 'Yes'].groupby('Zip Code').size()

# Convert index (zip codes) to integers to remove the .0 decimal part
church_prop_zip.index = church_prop_zip.index.astype(int)


In [87]:

results["How many churches owned vacant properties in the City of Dallas are in each zip code?"] = {
    'question_number': 33,
    'value': church_prop_zip
}

results["How many churches owned vacant properties in the City of Dallas are in each zip code?"] 

{'question_number': 33,
 'value': Zip Code
 75203    2
 75211    2
 75212    4
 75215    2
 75216    6
 75217    3
 75223    1
 75224    2
 75226    2
 75227    3
 75228    1
 75238    1
 75241    6
 dtype: int64}

In [88]:
#34 How many are in each council district

church_prop_district = vacant_lots_df[vacant_lots_df['Owner is Church'] == 'Yes'].groupby('Council District').size()


In [89]:

results["How many are in each council district"] = {
    'question_number': 34,
    'value': church_prop_district
}

results["How many are in each council district"]

{'question_number': 34,
 'value': Council District
 District 1     1
 District 2     1
 District 3     1
 District 4     8
 District 5     2
 District 6     4
 District 7    10
 District 8     7
 District 9     1
 dtype: int64}